### 복습 
- 가전 폴더 안에 모든 데이터파일을 로드해서 하나의 데이터프레임으로 생성 
- 감정에 대한 데이터들이 3개 분류 -> 2개의 분류로 변경 (부정, 중립 -> 부정)
- 감정 데이터가 없는 데이터들은 따로 저장 
- train, test의 비율은 8:2
- Dataset을 기존의 Dataset 구성과 같이 작업
- RawText 데이터를 이용하여 감정분석 모델을 생성 
- SBERT 모델을 이용하여 임베딩 
- 다중퍼셉트론의 모델을 이용하여 감정 분석 (Linear -> ReLU -> DropOut -> Linear)
- 검증 데이터를 이용하여 정확도와 f1_score 확인 
- 감정 데이터가 없는 RawText에서 sample(10)를 출력하여 감정 예측

- 다중퍼셉트론 모델이 아닌 머신러닝 모델 (SVC)을 이용하여 감정 분석 예측

In [ ]:
import os 
import re
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

In [ ]:
# 문자 정규화 함수 정의 
def normailize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

In [ ]:
# 파일의 목록을 로드 -> 목록을 기준으로 데이터를 로드 -> 단순 결합 
file_path = "../data/가전/"
file_list = os.listdir(file_path)
file_list

In [ ]:
# 로드한 데이터프레임을 누적으로 결합하기 위해 빈 데이터프레임 생성 
df = pd.DataFrame()

for file in file_list:
    # file : 파일명 
    data = pd.read_json(file_path + file)
    # df에 단순 행 결합 -> df에 다시 대입 
    df = pd.concat( [df, data], axis= 0  )
df.info()

In [ ]:
# 필요한 컬럼을 제외하고 나머지 컬럼을 무시 
df = df[['RawText', 'GeneralPolarity']]

In [ ]:
df.info()

In [9]:
df.rename(columns = {
    'GeneralPolarity' : 'label'
}, inplace=True)

In [10]:
# RawText의 문자 정규화 사용
df['RawText'] = df['RawText'].map(normailize)

In [11]:
# RawText에서 길이가 1 이하인 데이터를 제외
df = df.loc[df['RawText'].str.len() > 1]

In [13]:
# 혹시 RawText에 중복 데이터가 존재할수 있으니 중복 제거 
df.drop_duplicates(subset='RawText', inplace=True)

In [ ]:
# label이 결측치인 데이터는 따로 저장 
na_df = df.loc[df['label'].isna(), ]
na_df

In [15]:
# df에는 결측치를 제외
df = df.loc[ ~df['label'].isna(), ]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3678 entries, 0 to 99
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 86.2+ KB


In [16]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 57.6+ KB


In [17]:
# label의 데이터의 빈도수를 확인 
df['label'].value_counts()

label
 1.0    2220
 0.0     944
-1.0     514
Name: count, dtype: int64

In [19]:
# label의 데이터들을 int형태로 변경 
df['label'] = df['label'].astype(int)

In [20]:
df['label'].value_counts()

label
 1    2220
 0     944
-1     514
Name: count, dtype: int64

In [22]:
# 삼중 분류의 class를 이진 분류로 변경하기 위해 -1을 0으로 변경 
df['label'] = df['label'].map(
    {
        -1 : 0, 
        0 : 0, 
        1 : 1
    }
)

In [23]:
df['label'].value_counts()

label
1    2220
0    1458
Name: count, dtype: int64

In [24]:
# train, test 로 분할
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

In [ ]:
train_df['label'].value_counts()

In [26]:
model_name = 'BM-K/KoSimCSE-roberta-multitask'
sbert = SentenceTransformer(model_name)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [27]:
# 최대 토큰 길이 제한 
sbert.max_seq_length = 128

In [28]:
class SBERTHead(Dataset):
    def __init__(self, texts, labels):
        with torch.inference_mode():
            self.emb = sbert.encode(
                texts, convert_to_tensor=True, normalize_embeddings=True
            )
            self.labels = torch.tensor(labels, dtype=torch.long)
        self.texts = texts
        self.labels2 = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]
        # getitem에서 임베딩 처리 
        # res_emb = sbert.encode(
        #     self.texts[idx], convert_to_tensor=True, normalize_embeddings=True
        # )
        # res_label = torch.tensor(self.labels2[idx], dtype=torch.long)
        # return res_emb, res_label

In [29]:
train_ds = SBERTHead(train_df['RawText'].tolist(), train_df['label'].tolist())
test_ds = SBERTHead(test_df['RawText'].tolist(), test_df['label'].tolist())

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=256)

In [30]:
# 다중 퍼셉트론 구조의 분류 모델 생성 
class MLPHead(nn.Module):
    def __init__(self, input_dim, hidden = 256, num_classes = 2, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(), 
            nn.Dropout(dropout), 
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        result = self.net(x)
        return result

In [31]:
# sbert 모델에서 출력 차원의 개수를 확인
in_dim = sbert.get_sentence_embedding_dimension()
clf = MLPHead(in_dim)
# 손실 함수 
crit = nn.CrossEntropyLoss()
# AdamW -> Adam 개선판
opt = optim.AdamW(clf.parameters(), lr=2e-4)

In [40]:
clf.train()

for epoch in range(5):
    total = 0.0
    for x, y in train_dl:
        opt.zero_grad()
        logits = clf(x)
        loss = crit(logits, y)
        loss.backward()
        opt.step()
        total += loss.item() * x.size(0)
    print(f"epoch : {epoch}, loss : {round(total/len(train_df), 4)}")

epoch : 0, loss : 0.4786
epoch : 1, loss : 0.462
epoch : 2, loss : 0.4525
epoch : 3, loss : 0.4466
epoch : 4, loss : 0.4421


In [41]:
# 정확도, f1-score 확인 
clf.eval()

y_true , y_pred = [], []

with torch.inference_mode():
    for x, y in test_dl:
        logits = clf(x)
        pred = logits.argmax(dim = 1).tolist()
        y_true.extend(y.tolist())
        y_pred += pred


In [42]:

print('accuracy_score', accuracy_score(y_true, y_pred))
print('f1_score', f1_score(y_true, y_pred))

accuracy_score 0.7894021739130435
f1_score 0.8324324324324325


In [ ]:
samples = na_df['RawText'].sample(10).tolist()
samples

In [46]:
@torch.no_grad()
def predict_review(texts, batch_size = 128):
    if isinstance(texts, str):
        texts = [texts]
    texts_norm = [normailize(t) for t in texts]  

    sbert.eval()
    clf.eval()

    result = []

    for idx in range(0, len(texts), batch_size):
        batch_text = texts_norm[idx : idx + batch_size]
        embs = sbert.encode(
            batch_text, convert_to_tensor = True, 
            normalize_embeddings = True
        )

        logits = clf(embs)
        probs = logits.softmax(dim = -1)
        preds = probs.argmax(dim = -1).tolist()
        id2label = {
            0 : '부정', 
            1 : '긍정'
        }
        for idx2, pred in enumerate(preds):
            prob = float(probs[idx2, pred])
            review = texts[idx + idx2]
            label = id2label[pred]
            result.append(
                {
                    'text' : review, 
                    'prob' : prob, 
                    'label' : label
                }
            )
    return result

In [47]:
predict_review(samples)

[{'text': '명절 선물 드릴거라 제품은 아직 사용 안해 봤는데요와... OOO 한개 OOO 한개 샀는데 둘 다 사은품으로 준 헤어 에센스인가 줄줄 샜어요.한개는 안예 문앞에 택배 있을 때 부터 흘러나온 에센스 때문에 택배 상자가 바깥까지 젖었고 에센스 냄새가 폴폴 나고 있었고 열어보니 에센스 1 5 정도 겨우 남아있었고요 흘러나온 에센스때문에 드라이기 받침인가 그거까지 젖어있고요받자마자 미끌미끌 냄새나고 줄줄 흘러서 욕실에 갖다놓고 말았어요.속상해서 안 쳐다보게 되네요.다른 하나는 택배 상자 겉은 멀쩡하길래 다행이다 하고 열어보니 이것도 에센스 새서 미끌거리고요.상자도 젖어서 버렸고 선물 해야 해서 시간도 없으니 반품은 안하지만 참 이런 배송 정말 오랜만이에요. 화나네요. 선물받으신분이 제품 마감상태가 별로라고 하시네요 전에 쓰던것보다 무거워서 손목이 아프시대요',
  'prob': 0.7514744400978088,
  'label': '부정'},
 {'text': '저희 부부가 애주가라서.. 특히 포도주요 심플하고 모던한 디자인에 8병 들어가는 사이즈로 구매했어요 컴프레서를 사용하지 않은 반도체 방식 어쩌고라 진동 없는 거 확실하고 소음은 저소음이지만 저는 그리 예민하질 않아서 쓰는 사람 기준에 따라 다를 것이고 저는 거슬리는 것 없이 마음에 딱 들어요 그리고 와인셀러가 냉장고이니 냉장고로서의 기본만 충실하면 되는 것 설정온도 잘 유지 하는게 맛있는 포도주를 마시는 비법 플러스 조명등이 상당히 밝아서 조명등만 키면 문 열지 않아도 안이 훤히 보여 편리하네요',
  'prob': 0.8135926723480225,
  'label': '긍정'},
 {'text': '출장이 잦아서 구매했는데 생각보다 실망스럽네요.기대하면서 제품 받고 착용했는데 불편했던 점이 꽤나 있었습니다.우선은 버튼들이 모두 측면에 있어서 어디에 어떤 버튼이 있는지가 확실하게 감지가 안되더라구요. 익숙해지면 저절로 손이 가겠지만 초반에는 불편합니다. 그리고 가벼워서 그런지 진동이 전